In [2]:
from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
# Константы
img_side = 120

# Разбиваем датасет на тренировочную группу и группу валидации
def init_data_with_batch_size(batch_size):
    global train_data
    train_data = tf.keras.utils.image_dataset_from_directory(
        "flowers",
        image_size=(img_side, img_side),
        shuffle=True,
        batch_size=batch_size,
    )

    # Убираем лейблы (т.к. у нас задача не распознавать изображения)
    train_data = train_data.map(lambda x, y: (x/255.0, x/255.0))

    return train_data

Модель

In [25]:
from keras.layers import (Input, Dense, Conv2D, Conv2DTranspose, BatchNormalization, UpSampling2D,
                          Dropout, Flatten, Reshape, Lambda, MaxPool2D, Concatenate, add, Cropping2D)
from keras.models import Model
from keras.optimizers import RMSprop, Adam
from math import log2

# Константы
filters = 64
hidden_units = 1024  # Только степень 4 (4**?) (Размер "смыслового вектора")
amount_encode_layers = 6
amount_decode_layers = 3
max_boudle_residual_layers = 3  # Сколько слоёв с остаточноым обучением перед уменьшением/увеличением картинки
"""
filters                     Слабое  влияние на количество параметров
hidden_units                Никакое влияние на количество параметров
amount_encode_layers        Сильное влияние на количество параметров
amount_decode_layers        Сильное влияние на количество параметров
max_boudle_residual_layers  Слабое  влияние на количество параметров
"""

core_size = (3, 3)
input_img = Input(shape=(img_side, img_side, 3))

"""Энкодер"""
# Это надо чтобы первый слой мог сложиться со следующим (при помощи add([x_temp, x]) )
x_temp = BatchNormalization()(input_img)
x_temp = Conv2D(filters//2**amount_encode_layers, core_size, activation="relu", padding="same")(x_temp)

for i in range(amount_encode_layers, 0, -1):
    x_temp = BatchNormalization()(x_temp)
    for _ in range(max_boudle_residual_layers):
        x = Conv2D(filters // 2**i, core_size, activation="relu", padding="same")(x_temp)
        x = Conv2D(filters // 2**i, core_size, activation="relu", padding="same")(x)
        x_temp = add([x_temp, x])

    x_temp = MaxPool2D((2, 2))(x_temp)
    x_temp = Conv2D(filters // 2**(i - 1), core_size, activation="relu", padding="same")(x_temp)


# Превращаем сжатую картинку в "смысловой вектор"
x = Flatten()(x_temp)
x = Dense(hidden_units, activation="relu")(x)
x = Reshape((int(hidden_units**.5), int(hidden_units**.5), 1))(x)
x = BatchNormalization()(x)


"""Декодер"""
# Расширяем карту признаков, увеличиваем картинку, и остаточное обучение
for i in range(amount_decode_layers):
    x_temp = BatchNormalization()(x_temp)
    for _ in range(max_boudle_residual_layers):
        x = Conv2DTranspose(filters, core_size, activation="relu", padding="same")(x_temp)
        x = Conv2DTranspose(filters, core_size, activation="relu", padding="same")(x)
        x_temp = add([x_temp, x])

    x_temp = Conv2DTranspose(filters, core_size, activation="relu", padding="same", strides=2)(x_temp)

# Просто добавляем признаки, не меняя картинку, но уменьшая колчество фильтров до 1
# (чтобы в Dense поступало не так много данных)
for i in range(int(log2(filters))):
    x_temp = BatchNormalization()(x_temp)
    for _ in range(max_boudle_residual_layers):
        x = Conv2DTranspose(filters // 2**i, core_size, activation="relu", padding="same")(x_temp)
        x = Conv2DTranspose(filters // 2**i, core_size, activation="relu", padding="same")(x)
        x_temp = add([x_temp, x])

    # Чтобы количество фильтров совпадало (для остаточного обучения)
    x_temp = Conv2DTranspose(filters // 2**(i +1), core_size, activation="relu", padding="same")(x_temp)

# Увеличиваем чёткость при помощи Dense на последнем слое
shape = x_temp.shape[1]
x = Flatten()(x_temp)
x = Dense(img_side**2, activation="relu")(x)
x = Reshape((img_side, img_side, 1))(x)

# Добавляем цвета
x = Conv2D(3, core_size, activation="sigmoid", padding="same")(x)
x = Conv2D(3, core_size, activation="sigmoid", padding="same")(x)
output_img = Reshape((img_side, img_side, 3))(x)

# Модель (ебать, я тут до 125 слоёв делал, и оно работало!)
vae = Model(input_img, output_img, name="vae")
vae.compile(
    optimizer=keras.optimizers.Adam(2e-3),
    loss="binary_crossentropy",
    loss_weights=[1000],
)

vae.summary()

Model: "vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization_741 (Batch  (None, 120, 120, 3)  12         ['input_19[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 conv2d_709 (Conv2D)            (None, 120, 120, 1)  28          ['batch_normalization_741[0][0]']
                                                                                                

In [ ]:
train_data = init_data_with_batch_size(4)
vae.fit(
    train_data,
    epochs=200,
)

Found 1671 files belonging to 5 classes.
Epoch 1/200
418/418 [==============================] - 15s 27ms/step - loss: 648.8441
Epoch 2/200
275/418 [==================>...........] - ETA: 3s - loss: 646.2441

In [ ]:
import matplotlib.pyplot as plt
from random import randint
import numpy as np

data = np.array([i[0][0] for count, i in enumerate(init_data_with_batch_size(1))
                 if count < 16])
generated_images = vae.predict(data, verbose=False)

num_images = 5

plt.figure(figsize=(20, 10))

for _ in range(num_images):
    random_num = randint(0, 16-1)

    # Оригинальное изображение
    plt.subplot(2, num_images, _ + 1)
    plt.imshow(data[random_num])
    plt.title("Original")
    plt.axis("off")

    # Сгенерированное изображение
    plt.subplot(2, num_images, _ + num_images + 1)
    plt.imshow(generated_images[random_num])
    plt.title("Generated")
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
"""Выводим Архитектуру"""
img_file = "architecture.png"
tf.keras.utils.plot_model(vae, to_file=img_file, show_shapes=True, show_layer_names=False)